# Homework 1

First, make sure all needed packages are installed:


In [1]:
pip install pymystem3 nltk pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## 1. Read the novel text from txt file

Open the txt file, read from it and then close it.

In [2]:
f = open("./goncharov_obryv.txt", "r")
novel = f.read()
f.close()

Print first 500 characters to make sure everything is fine:

In [3]:
print(novel[:500])

Иван Александрович Гончаров

Обрыв

Роман в пяти частях.



ЧАСТЬ ПЕРВАЯ





I





Два господина сидели в небрежно убранной квартире в Петербурге, на одной из больших улиц. Одному было около тридцати пяти, а другому около сорока пяти лет.

Первый был Борис Павлович Райский, второй -- Иван Иванович Аянов.

У Бориса Павловича была живая, чрезвычайно подвижная физиономия. С первого взгляда он казался моложе своих лет: большой белый лоб блистал свежестью, глаза менялись, то загорались мыслию, чувс


---
## 2. Lemmatize text using Mystem and save it

First, import Mystem and create an instance of Mystem class:

In [4]:
from pymystem3 import Mystem

m = Mystem()

Lemmatize the text, and print a short slice of the obtained lemmas list:

In [5]:
lemmas = m.lemmatize(novel)

In [6]:
lemmas[50:60]

['квартира', ' ', 'в', ' ', 'петербург', ', ', 'на', ' ', 'один', ' ']

Now we need to save this lemmatization into the new txt file. I'm not sure how exactly it should be formatted, so I will just join these normal forms of words into the new text and save it like that.

Open the file for writing, write into it and close it:

In [7]:
f = open("lemmatized_text.txt", "w")
f.write(''.join(lemmas))
f.close()

The saved text looks like this:

In [8]:
print(''.join(lemmas)[84:284])

два господин сидеть в небрежно убирать квартира в петербург, на один из большой улица. один быть около тридцать пять, а другой около сорок пять год.

первый быть борис павлович райский, второй -- иван


---
## 3 Tokenizing the text with NLTK and use pymorphy to analyze words

First, import NLTK.


In [9]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Tokenize the text and print a short slice of the obtained token list:

In [10]:
tokens = nltk.word_tokenize(novel)

In [11]:
tokens[100:110]

['.', 'Иногда', 'же', 'смотрели', 'они', 'зрело', ',', 'устало', ',', 'скучно']

Now let's analyze these tokens with pymorphy. Import the library and create an instance of it:

In [12]:
import pymorphy2 as mph

morph = mph.MorphAnalyzer()

Here I define a function `token_to_dict` that analyzes one token and returns a dictionary for it in desired format, and then create a list of such dictionaries by applying this function to each token. 



In [13]:
def token_to_dict(token):
  analysis = morph.parse(token)[0] # take the analysis with the highest score
  return {"lemma": analysis.normal_form, "word": analysis.word, "pos": analysis.tag.POS}

In [14]:
tokens_analysis = list(map(token_to_dict, tokens))

Now is good time to get rid of all punctuation, as all such tokens have their POS value equal to `None` and can easily be filtered out. I also count the number of tokens before and after the filtering.

In [15]:
len(tokens_analysis)

306236

In [16]:
tokens_analysis = list(filter(lambda tkn: tkn["pos"] != None, tokens_analysis))

In [17]:
len(tokens_analysis)

226709

We can see that approximately 80,000 tokens were filtered out. Here is a short slice of the obtaned array of dicts:

In [18]:
tokens_analysis[100:110]

[{'lemma': 'этот', 'word': 'эти', 'pos': 'ADJF'},
 {'lemma': 'неизгладимый', 'word': 'неизгладимые', 'pos': 'ADJF'},
 {'lemma': 'знак', 'word': 'знаки', 'pos': 'NOUN'},
 {'lemma': 'время', 'word': 'времени', 'pos': 'NOUN'},
 {'lemma': 'и', 'word': 'и', 'pos': 'CONJ'},
 {'lemma': 'опыт', 'word': 'опыта', 'pos': 'NOUN'},
 {'lemma': 'гладкий', 'word': 'гладкие', 'pos': 'ADJF'},
 {'lemma': 'чёрный', 'word': 'чёрные', 'pos': 'ADJF'},
 {'lemma': 'волос', 'word': 'волосы', 'pos': 'NOUN'},
 {'lemma': 'падать', 'word': 'падали', 'pos': 'VERB'}]

Now let's create a `.jsonl` file and save this information there. I import `json` library and use `json.dumps` to turn dicts into JSON strings.

In [19]:
import json

In [20]:
f = open("analyzed_tokens.jsonl", "w", encoding="utf-8")
f.write('\n'.join(json.dumps(tkn, ensure_ascii=False) for tkn in tokens_analysis))
f.close()

---
## 4.1. Calculating the shares of each part of speech

To do so, I first extact a list of all unique values of POS by using `set`. Then, for each part of speech, I calculate its share by filtering only tokens of this POS, counting them, and dividing their amount by the overall token amount.

In [21]:
parts_of_speech = set(tkn["pos"] for tkn in tokens_analysis)

for pos in parts_of_speech:
  share = len(list(filter(lambda tkn: tkn["pos"] == pos, tokens_analysis))) / len(tokens_analysis) * 100
  print(pos + ":\t" + f"{share:.2f}" + "%")

NUMR:	0.45%
NPRO:	11.71%
GRND:	1.21%
PRTS:	0.18%
PRCL:	6.58%
PRED:	0.65%
NOUN:	22.02%
PRTF:	0.66%
INFN:	2.79%
VERB:	15.38%
ADJS:	0.86%
CONJ:	11.15%
INTJ:	0.13%
ADVB:	6.90%
COMP:	0.40%
PREP:	10.09%
ADJF:	8.85%


We can see that the most frequent categories are Nouns (22%), Verbs (15%), Pronouns (11%), Conjunctives (11%) and Prepositions (10%).

---
## 4.2. Most frequent Verbs and Adverbs

To obtain the top-20 most frequent Verbs and Adverbs, I will use the `Counter` class.

In [22]:
from collections import Counter

First, I construct the `verbs` list, which contains all verb lemmas (with repititions).


In [23]:
verbs = [tkn["lemma"] for tkn in tokens_analysis if tkn["pos"] == "VERB"]

The `Counter` class allows to obtain two lists: `Counter(verbs).keys()` is a list of unique verb lemmas, `Counter(verbs).values()` is a list of the same size that for each unique lemma contains number of its occurencies in the `verbs` list. 

Then, I sort `Counter(verbs).keys()` in the descending order of corresponding values in `Counter(verbs).values()` by using `zip`. I obtain `sorted_verbs` -- a list of tuples of lemmas and number of their occurencies, sorted such that the most frequent ones are at the top.

In [24]:
sorted_verbs = sorted(zip(Counter(verbs).values(), Counter(verbs).keys()), reverse=True)

Print the top-20 verbs:

In [25]:
for count, lemma in sorted_verbs[:20]:
  print(lemma + "\t" + str(count) + " occurencies")

быть	1938 occurencies
сказать	1412 occurencies
говорить	866 occurencies
хотеть	614 occurencies
знать	606 occurencies
мочь	557 occurencies
спросить	427 occurencies
любить	415 occurencies
стать	388 occurencies
видеть	386 occurencies
думать	354 occurencies
пойти	321 occurencies
дать	275 occurencies
смотреть	258 occurencies
сделать	242 occurencies
идти	218 occurencies
заметить	217 occurencies
казаться	205 occurencies
взять	190 occurencies
делать	182 occurencies


All the same for the adverbs.

In [26]:
adverbs = [tkn["lemma"] for tkn in tokens_analysis if tkn["pos"] == "ADVB"]
sorted_adverbs = sorted(zip(Counter(adverbs).values(), Counter(adverbs).keys()), reverse=True)

In [27]:
for count, lemma in sorted_adverbs[:20]:
  print(lemma + "\t" + str(count) + " occurencies")

только	775 occurencies
ещё	627 occurencies
потом	534 occurencies
опять	509 occurencies
теперь	440 occurencies
вдруг	386 occurencies
там	379 occurencies
ничего	375 occurencies
где	317 occurencies
почти	252 occurencies
тут	237 occurencies
зачем	224 occurencies
уже	212 occurencies
уж	196 occurencies
здесь	188 occurencies
никогда	181 occurencies
вон	176 occurencies
тогда	170 occurencies
иногда	166 occurencies
тихо	159 occurencies


---
## 5. Bigrams and Trigrams

Since we already got rid of punctuation in `tokens_analysis`, we can extract the `lemma` column from there to get the list of lemmas that is cleared from punctuation.

In [28]:
lemmas = [tkn["lemma"] for tkn in tokens_analysis]
lemmas[100:110]

['этот',
 'неизгладимый',
 'знак',
 'время',
 'и',
 'опыт',
 'гладкий',
 'чёрный',
 'волос',
 'падать']

Get the list of bigrams using NLTK:

In [29]:
bigrams = list(nltk.bigrams(lemmas))

Sort them exactly in the same way, as in top-20 verbs and adverbs:

In [30]:
sorted_bigrams = sorted(zip(Counter(bigrams).values(), Counter(bigrams).keys()), reverse=True)

In [31]:
for count, bigram in sorted_bigrams[:25]:
  print("{0:20} \t {1} occurencies".format(" ".join(bigram), str(count)))
  

я не                 	 415 occurencies
и не                 	 413 occurencies
татьяна маркович     	 377 occurencies
сказать она          	 326 occurencies
она и                	 306 occurencies
она не               	 305 occurencies
у он                 	 295 occurencies
сказать он           	 278 occurencies
он не                	 272 occurencies
не знать             	 272 occurencies
на он                	 254 occurencies
что я                	 253 occurencies
не быть              	 253 occurencies
у она                	 245 occurencies
он и                 	 243 occurencies
не мочь              	 239 occurencies
как будто            	 225 occurencies
что он               	 222 occurencies
он в                 	 220 occurencies
на она               	 219 occurencies
что вы               	 207 occurencies
что она              	 204 occurencies
глядеть на           	 201 occurencies
и в                  	 186 occurencies
у я                  	 185 occurencies


The same for trigrams.

In [32]:
trigrams = list(nltk.trigrams(lemmas))
sorted_trigrams = sorted(zip(Counter(trigrams).values(), Counter(trigrams).keys()), reverse=True)

for count, trigram in sorted_trigrams[:25]:
  print("{0:20} \t {1} occurencies".format(" ".join(trigram), str(count)))

глядеть на он        	 63 occurencies
в сам дело           	 58 occurencies
не знать что         	 49 occurencies
глядеть на она       	 47 occurencies
я не хотеть          	 32 occurencies
я не знать           	 31 occurencies
что же вы            	 31 occurencies
татьяна маркович и   	 31 occurencies
на другой день       	 31 occurencies
я ничего не          	 29 occurencies
она за рука          	 29 occurencies
если б я             	 29 occurencies
у он в               	 27 occurencies
сказать она и        	 27 occurencies
что она не           	 26 occurencies
и не мочь            	 26 occurencies
сказать татьяна маркович 	 24 occurencies
не глядеть на        	 24 occurencies
взглянуть на он      	 24 occurencies
в этот минута        	 24 occurencies
поглядеть на он      	 23 occurencies
что я не             	 22 occurencies
она не быть          	 22 occurencies
она и не             	 22 occurencies
он глядеть на        	 22 occurencies


**Some commentaries:** We can see that the majority of bi- and trigrams contain some pronoun (я, вы, он, она,..) as they are very frequentive.

They are often combined with some preposition (на, у, в,..), which are also very frequentive, or with particle не.


---
## 6. Altering morphological parameters

Let's take the following 5 sentences:

In [69]:
sentence1 = novel[1321:1392]
print(sentence1)

Райский одет был в домашнее серенькое пальто, сидел с ногами на диване.


In [70]:
sentence2 = novel[7326:7348]
print(sentence2)

Так грозил ему доктор.


In [71]:
sentence3 = novel[10082:10119]
print(sentence3)

А вот с женщиной биться зиму и весну!


In [72]:
sentence4 = novel[20158:20226]
print(sentence4)

Ему дали отличную квартиру, лошадей, экипаж и тысяч двадцать дохода.


In [73]:
sentence5 = novel[21500:21712]
print(sentence5)

Он с наслаждением и завистью припоминал анекдоты времен революции, как один знатный повеса разбил там чашку в магазине и в ответ на упреки купца перебил и переломал еще множество вещей и заплатил за весь магазин;


The following function takes a string, and some token-modifying function. It tokenizes the string, applies all of the token-modifying functions, and then puts the string back together.

In [68]:
def alter_morphology(sentence, *rules):
  tokens = nltk.word_tokenize(sentence)
  tokens_parse = [morph.parse(tkn)[0] for tkn in tokens]

  result = ""
  for tkn, original_tkn in zip(tokens_parse, tokens):
    for rule in rules:
      tkn = rule(tkn)

    if result != "" and not mph.shapes.is_punctuation(tkn.word): result += " "
    result += mph.shapes.restore_capitalization(tkn.word, original_tkn)
  return result

Now let's define some token modifying operations, such as number inversion (singular to plural and plural to singular):

In [74]:
def number_invert(tkn):
  if 'sing' in tkn.tag and tkn.inflect({'plur'}) != None:
    return tkn.inflect({'plur'})
  if 'plur' in tkn.tag and tkn.inflect({'sing'}) != None:
    return tkn.inflect({'sing'})
  return tkn

Turning to a past tense:

In [76]:
def past(tkn):
  if tkn.inflect({'past'}) != None:
    return tkn.inflect({'past'})
  return tkn

Apply this operations to our sentences:

In [80]:
print(sentence1, alter_morphology(sentence1, number_invert), sep='\n')

Райский одет был в домашнее серенькое пальто, сидел с ногами на диване.
Райские одеты были в домашние серенькие пальто, сидели с ногой на диванах.


In [85]:
print(sentence5, alter_morphology(sentence5, number_invert, past), sep='\n')

Он с наслаждением и завистью припоминал анекдоты времен революции, как один знатный повеса разбил там чашку в магазине и в ответ на упреки купца перебил и переломал еще множество вещей и заплатил за весь магазин;
Он с наслаждениями и завистями припоминали анекдот времени революций, как одни знатные повесы разбили там чашки в магазинах и в ответы на упрёк купцов перебили и переломали ещё множества вещи и заплатили за все магазины;


In [71]:
stokens_parse = [morph.parse(tkn)[0] for tkn in nltk.word_tokenize(sentence3)]
stokens_parse

[Parse(word='а', tag=OpencorporaTag('CONJ'), normal_form='а', score=0.997155, methods_stack=((DictionaryAnalyzer(), 'а', 20, 0),)),
 Parse(word='вот', tag=OpencorporaTag('PRCL'), normal_form='вот', score=1.0, methods_stack=((DictionaryAnalyzer(), 'вот', 22, 0),)),
 Parse(word='с', tag=OpencorporaTag('PREP'), normal_form='с', score=0.997625, methods_stack=((DictionaryAnalyzer(), 'с', 393, 0),)),
 Parse(word='женщиной', tag=OpencorporaTag('NOUN,anim,femn sing,ablt'), normal_form='женщина', score=1.0, methods_stack=((DictionaryAnalyzer(), 'женщиной', 53, 4),)),
 Parse(word='биться', tag=OpencorporaTag('INFN,impf,intr'), normal_form='биться', score=1.0, methods_stack=((DictionaryAnalyzer(), 'биться', 466, 0),)),
 Parse(word='зиму', tag=OpencorporaTag('NOUN,inan,femn sing,accs'), normal_form='зима', score=1.0, methods_stack=((DictionaryAnalyzer(), 'зиму', 55, 3),)),
 Parse(word='и', tag=OpencorporaTag('CONJ'), normal_form='и', score=0.998263, methods_stack=((DictionaryAnalyzer(), 'и', 20, 0